In [ ]:
!pip install opencv-python mediapipe matplotlib numpy pycaw pygame mido python-rtmidi tensorflow

In [25]:
import cv2
import mediapipe as mp
import time
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

def calculate_fingertips_distance_matrix(landmarks, img_shape, indices=[4, 8, 12, 16, 20]):
    """
    Oblicza znormalizowaną macierz odległości między końcówkami palców.
    
    Parameters:
        landmarks: Lista landmarków dłoni z Mediapipe.
        img_shape: Rozmiar obrazu (do normalizacji).
        indices: Indeksy końcówek palców.
    
    Returns:
        Znormalizowana macierz odległości (np. 5x5 dla 5 końcówek palców).
    """
    selected_points = [landmarks[i] for i in indices]
    num_points = len(selected_points)
    distance_matrix = np.zeros((num_points, num_points))
    
    for i in range(num_points):
        for j in range(num_points):
            x1, y1 = selected_points[i].x * img_shape[1], selected_points[i].y * img_shape[0]
            x2, y2 = selected_points[j].x * img_shape[1], selected_points[j].y * img_shape[0]
            distance_matrix[i, j] = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    # Normalizacja macierzy
    max_dist = np.max(distance_matrix) if np.max(distance_matrix) > 0 else 1
    distance_matrix /= max_dist
    
    return distance_matrix

def get_hand_position_normalized(landmarks):
    x_positions = [lm.x for lm in landmarks]
    return np.mean(x_positions)


def draw_heatmap(img, matrix, position=(10, 10), size=(150, 150)):
    normalized_matrix = (matrix * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(cv2.resize(normalized_matrix, size, interpolation=cv2.INTER_NEAREST), cv2.COLORMAP_VIRIDIS)
    x, y = position
    img[y:y+size[1], x:x+size[0]] = heatmap
    return img


def embed_states_on_image(img, states):
    """
    Wizualizuje stany klawiszy MIDI na obrazie.

    Parametry:
        img (np.ndarray): Obraz wejściowy.
        states (np.ndarray): Wektor stanów MIDI (256 elementów).

    Zwraca:
        np.ndarray: Obraz z nałożonymi stanami.
    """
    h, w, _ = img.shape
    for i, state in enumerate(states):
        x = int((i / 128) * 600)
        y = 256
        if state > 0.5:
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        else:
            cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
            
    return img

def parse_midi_file(filename):
    """
    Parsuje plik MIDI stanu.

    Parametry:
        filename (str): Ścieżka do pliku MIDI.

    Zwraca:
        np.ndarray: Tablica 2D, gdzie każdy wiersz odpowiada stanowi klawiszy dla jednej klatki.
    """
    states = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("Frame"):
                state_str = line.split(":")[1].strip().strip("[]")
                state = [int(x) for x in state_str.split(",")]
                states.append(state)
    return np.array(states)


In [397]:
from keras.layers import Input, Dense, Flatten, Lambda, Concatenate
from keras.models import Model
import tensorflow as tf

# Stałe
NUM_KEYS = 128  # Liczba klawiszy
FEATURE_SIZE = 10  # Liczba cech z sigmoid (wciśnięcia klawiszy)

# Funkcja aktualizująca wektor stanów
def update_key_states(features, position, num_keys=NUM_KEYS):
    def scatter_update(inputs):
        features, position = inputs
        batch_size = tf.shape(features)[0]

        # Obliczenie indeksu startowego
        start_idx = tf.cast((position * tf.cast(num_keys, tf.float32)) - (FEATURE_SIZE / 2.0), tf.int32)
        start_idx = tf.reshape(start_idx, [-1])  # (batch_size,)

        # Przygotowanie pustego wektora stanów
        key_states = tf.zeros([batch_size, num_keys], dtype=tf.float32)

        # Generowanie indeksów i aktualizacja stanu
        for i in range(FEATURE_SIZE):
            idx = start_idx + i  # Aktualna pozycja
            idx = tf.clip_by_value(idx, 0, num_keys - 1)  # Indeksy w zakresie [0, num_keys - 1]
            
            # Tworzenie scatter_indices o kształcie (batch_size, 2)
            scatter_indices = tf.stack([tf.range(batch_size), idx], axis=1)
            
            # Dodanie wartości do wektora stanów
            key_states = tf.tensor_scatter_nd_add(
                key_states,
                scatter_indices,
                features[:, i]  # Aktualne wartości do dodania
            )
        return key_states

    return Lambda(
        scatter_update,
        output_shape=(NUM_KEYS,)  # Wyjście ma kształt (batch_size, num_keys)
    )([features, position])

# Wejścia
input_handl_x = Input(shape=(1,), name="HANDL_X", dtype=tf.float32)  # Pozycja lewej dłoni
input_handr_x = Input(shape=(1,), name="HANDR_X", dtype=tf.float32)  # Pozycja prawej dłoni
input_handl_matrix = Input(shape=(5, 5), name="HANDL_MATRIX", dtype=tf.float32)  # Macierz lewej dłoni
input_handr_matrix = Input(shape=(5, 5), name="HANDR_MATRIX", dtype=tf.float32)  # Macierz prawej dłoni

# Przetwarzanie macierzy dłoni
flatten_handl_matrix = Flatten()(input_handl_matrix)
flatten_handr_matrix = Flatten()(input_handr_matrix)

flatten_handl_matrix = Concatenate(axis=-1)([flatten_handl_matrix, input_handl_x])
flatten_handr_matrix = Concatenate(axis=-1)([flatten_handr_matrix, input_handr_x])

#dense_handl_matrix = Dense(25, activation='relu')(flatten_handl_matrix)
#dense_handr_matrix = Dense(25, activation='relu')(flatten_handr_matrix)

dense_handl_matrix = Dense(128, activation='relu', name="HANDL_FEATURES")(flatten_handl_matrix)
dense_handr_matrix = Dense(128, activation='relu', name="HANDR_FEATURES")(flatten_handr_matrix)

# Aktualizacja stanów klawiszy na podstawie pozycji dłoni i cech
#ey_states_l = update_key_states(dense_handl_matrix, input_handl_x)
#key_states_r = update_key_states(dense_handr_matrix, input_handr_x)

# Połączenie wyników z obu dłoni
#combined_states = Add(name="COMBINED_STATES")([key_states_l, key_states_r])
combined_states = Add(name="COMBINED_STATES")([dense_handl_matrix, dense_handr_matrix])

# Model
model = Model(
    inputs=[input_handl_x, input_handr_x, input_handl_matrix, input_handr_matrix],
    outputs=combined_states
)

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Podsumowanie modelu
model.summary()


Model: "functional_49"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ HANDL_MATRIX        │ (None, 5, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ HANDR_MATRIX        │ (None, 5, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_114         │ (None, 25)        │          0 │ HANDL_MATRIX[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ HANDL_X             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_115         │ (None, 25)        │          0 │ HANDR_MATRIX[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ HANDR_X             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_20      │ (None, 26)        │          0 │ flatten_114[0][0… │
│ (Concatenate)       │                   │            │ HANDL_X[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_21      │ (None, 26)        │          0 │ flatten_115[0][0… │
│ (Concatenate)       │                   │            │ HANDR_X[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ HANDL_FEATURES      │ (None, 128)       │      3,456 │ concatenate_20[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ HANDR_FEATURES      │ (None, 128)       │      3,456 │ concatenate_21[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ COMBINED_STATES     │ (None, 128)       │          0 │ HANDL_FEATURES[0… │
│ (Add)               │                   │            │ HANDR_FEATURES[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,912 (27.00 KB)

 Trainable params: 6,912 (27.00 KB)

 Non-trainable params: 0 (0.00 B)

In [220]:

# Wczytanie danych z plików
cap = cv2.VideoCapture("C:\\Users\\Malak\\data2\\recording.avi")
midi_states = parse_midi_file("C:\\Users\\Malak\\data2\\midi_state.txt")

hand_positions_l = []
hand_positions_r = []
hand_matrices_l = []
hand_matrices_r = []
output_data = []  # Lista na dane wyjściowe (stany MIDI)

mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2)
mpDraw = mp.solutions.drawing_utils

frame_idx = 0
while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    hand_positions = [0, 0]  # Domyślne wartości dla pozycji dłoni
    hand_matrices = [np.zeros((5, 5)), np.zeros((5, 5))]  # Domyślne wartości dla macierzy dłoni

    if results.multi_hand_landmarks:
        for idx, handLms in enumerate(results.multi_hand_landmarks):
            if idx > 1:
                break  # Obsługujemy maksymalnie 2 dłonie
            if idx == 0:
                hand_positions[0] = get_hand_position_normalized(handLms.landmark)
                hand_matrices[0] = calculate_fingertips_distance_matrix(handLms.landmark, img.shape)
            else:
                hand_positions[1] = get_hand_position_normalized(handLms.landmark)
                hand_matrices[1] = calculate_fingertips_distance_matrix(handLms.landmark, img.shape)
    
    hand_positions_l.append(hand_positions[0])
    hand_positions_r.append(hand_positions[1])
    hand_matrices_l.append(hand_matrices[0])
    hand_matrices_r.append(hand_matrices[1])
    output_data.append(midi_states[frame_idx])
    frame_idx += 1

    if frame_idx >= len(midi_states):
        break

cap.release()

In [238]:
# Przygotowanie danych do treningu
x_handl_x = np.array(hand_positions_l).reshape(-1, 1)
x_handr_x = np.array(hand_positions_r).reshape(-1, 1)
x_handl_matrix = np.stack(hand_matrices_l)
x_handr_matrix = np.stack(hand_matrices_r)
output_data = np.array(output_data)

print(len(x_handl_x), len(x_handr_x), len(x_handl_matrix), len(x_handr_matrix), len(output_data))

# Sprawdzenie wymiarów danych
print(np.array(x_handl_x).shape)
print(np.array(x_handr_x).shape)
print(np.array(x_handl_matrix).shape)
print(np.array(x_handr_matrix).shape)
print(np.array(output_data).shape)

1833 1833 1833 1833 1833
(1833, 1)
(1833, 1)
(1833, 5, 5)
(1833, 5, 5)
(1833, 128)


In [ ]:
# Trening modelu
model.fit([x_handl_x, x_handr_x, x_handl_matrix, x_handr_matrix], output_data, epochs=660, batch_size=48)
#class_weight={0: 0.078125, 1: 0.921875}

In [35]:
import cv2
import numpy as np
import mediapipe as mp
import pygame.midi

# Inicjalizacja pygame.midi
pygame.midi.init()

# Wyświetlenie dostępnych urządzeń MIDI
for i in range(pygame.midi.get_count()):
    info = pygame.midi.get_device_info(i)
    print(f"ID: {i}, Nazwa: {info[1]}, Typ: {'Wyjście' if info[3] else 'Wejście'}")


player = pygame.midi.Output(0)
player.set_instrument(0)  # Ustawienie na standardowy fortepian MIDI

# Inicjalizacja MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2)
mpDraw = mp.solutions.drawing_utils

# Mapowanie końcówek palców na wektor 128
def map_fingertips_to_vector(fingertips, vector_size=128):
    vector = np.zeros(vector_size)
    for tip in fingertips:
        index = int(tip * vector_size)  # Mapowanie pozycji na indeks
        index = np.clip(index, 0, vector_size - 1)  # Ograniczenie zakresu
        vector[index] = 1
    return vector

# Funkcja wizualizacji wektora na obrazie
def embed_states_on_image(image, states):
    h, w, _ = image.shape
    overlay = image.copy()
    for i, state in enumerate(states):
        x = int(i / 128 * w)  # Pozycja na obrazie
        if state == 1:
            cv2.line(overlay, (x, h-120), (x, h - (120+50)), (0, 255, 0), 2)
        else: 
            cv2.line(overlay, (x, h-120), (x, h - (120+50)), (0, 0, 255), 2)
        
    return cv2.addWeighted(overlay, 0.7, image, 0.3, 0)

def calculate_normalized_distance_matrix(landmarks, img_shape):
    """
    Oblicza znormalizowaną macierz odległości dla podanych punktów odniesienia dłoni.

    Parametry:
        landmarks (list): Lista punktów odniesienia (landmarków) dłoni.
        img_shape (tuple): Wymiary obrazu (wysokość, szerokość, kanały).

    Zwraca:
        np.ndarray: Znormalizowana macierz odległości między wszystkimi parami punktów.
    """
    h, w, _ = img_shape
    points = np.array([[lm.x * w, lm.y * h] for lm in landmarks], dtype=np.float32)

    distances = np.linalg.norm(points[:, None, :] - points[None, :, :], axis=-1)

    max_distance = np.max(distances)
    if max_distance > 0:
        normalized_distances = distances / max_distance
    else:
        normalized_distances = distances

    return normalized_distances

# Funkcja odtwarzania dźwięków MIDI
active_notes = set()  # Przechowuje aktywne nuty
def play_midi_notes(vector):
    global active_notes
    new_active_notes = set(np.where(vector == 1)[0])  # Aktywne nuty z wektora

    # Graj nowe nuty
    for note in new_active_notes - active_notes:
        player.note_on(note, 127)  # Graj nutę z maksymalną siłą (127)

    # Zatrzymaj nuty, które już nie są aktywne
    for note in active_notes - new_active_notes:
        player.note_off(note, 127)  # Zatrzymaj nutę

    # Zaktualizuj aktywne nuty
    active_notes = new_active_notes

# Główna pętla
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    output_vector = np.zeros(128)  # Wektor wyjściowy (128 klawiszy)
    hand_positions = [0, 0]
    hand_matrices = [np.zeros((5, 5)), np.zeros((21, 21))]


    
    if results.multi_hand_landmarks:
        for idx, handLms in enumerate(results.multi_hand_landmarks):
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

            # Pobieranie pozycji końcówek palców
            fingertips_indices = [4, 8, 12, 16, 20]  # Landmarki końcówek palców
            fingertips = [handLms.landmark[tip].x for tip in fingertips_indices]  # Pozycje X (0-1)

            # Mapowanie końcówek palców na wektor 128
            hand_vector = map_fingertips_to_vector(fingertips)

            # Aktualizacja wektora wyjściowego
            output_vector = np.maximum(output_vector, hand_vector)

            # Rysowanie końcówek palców
            h, w, c = img.shape
            for tip in fingertips_indices:
                x = int(handLms.landmark[tip].x * w)
                y = int(handLms.landmark[tip].y * h)
                cv2.circle(img, (x, y), 10, (0, 255, 122), cv2.FILLED)

            if idx > 1:
                break
            if idx == 0:
                hand_positions[0] = get_hand_position_normalized(handLms.landmark)
                hand_matrices[0] = calculate_normalized_distance_matrix(handLms.landmark, img.shape)
            else:
                hand_positions[1] = get_hand_position_normalized(handLms.landmark)
                hand_matrices[1] = calculate_normalized_distance_matrix(handLms.landmark, img.shape)

            # Rysowanie macierzy odległości dla lewej dłoni
            img = draw_heatmap(img, hand_matrices[0], position=(10, 10))
            # Rysowanie macierzy odległości dla prawej dłoni
            img = draw_heatmap(img, hand_matrices[1], position=(170, 10))

    # Wizualizacja wektora na obrazie
    img = embed_states_on_image(img, output_vector)

    # Odtwarzanie dźwięków MIDI
    play_midi_notes(output_vector)

    # Wyświetlanie obrazu
    cv2.imshow("Hand Tracking", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


ID: 0, Nazwa: b'Microsoft MIDI Mapper', Typ: Wyjście
ID: 1, Nazwa: b'Roland Digital Piano', Typ: Wejście
ID: 2, Nazwa: b'Microsoft GS Wavetable Synth', Typ: Wyjście
ID: 3, Nazwa: b'Roland Digital Piano', Typ: Wyjście


KeyboardInterrupt: 

In [38]:
cap.release()
cv2.destroyAllWindows()
player.close()
pygame.midi.quit()

In [54]:
import cv2
import time
from mido import MidiFile, MidiTrack, Message
import mido
import os
import numpy as np

# Funkcja do nagrywania wideo i MIDI jednocześnie
def record_video_and_midi(output_video_path, output_midi_state_path):
    print("Rozpoczynanie nagrywania wideo i MIDI...")
    
    # Przygotowanie kamery
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Nie można otworzyć kamery")
        return

    # Parametry wideo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30

    # Ustawienie kodeka i obiektu zapisu wideo
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Lista do przechowywania historii klatek i stanu MIDI
    midi_state_history = []

    # Bieżący stan MIDI
    current_midi_state = [0] * 128  # 128 klawiszy MIDI

    # Otwórz port MIDI
    with mido.open_input() as port:
        start_time = time.time()
        frame_idx = 0

        while True:
            # Pobierz klatkę z kamery
            ret, frame = cap.read()
            if not ret:
                print("Nie można odczytać klatki")
                break

            # Aktualizuj stan MIDI
            for msg in port.iter_pending():
                if msg.type == 'note_on' and msg.velocity > 0:
                    current_midi_state[msg.note] = 1  # Klawisz wciśnięty
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    current_midi_state[msg.note] = 0  # Klawisz zwolniony

            # Zapisz klatkę i stan MIDI tylko jeśli przynajmniej jeden klawisz jest wciśnięty
            if any(current_midi_state):
                # Wyświetl podgląd wideo
                cv2.imshow('Nagrywanie', frame)

                # Zapisz klatkę do pliku wideo
                out.write(frame)

                # Zapisz stan MIDI dla tej klatki
                midi_state_history.append((frame_idx, current_midi_state.copy()))
                frame_idx += 1

            # Sprawdź, czy użytkownik chce zakończyć nagrywanie
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Zakończ nagrywanie
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Zapisz historię stanu MIDI
    with open(output_midi_state_path, "w") as f:
        for frame_idx, midi_state in midi_state_history:
            f.write(f"Frame {frame_idx}: {midi_state}\n")
    print(f"Historia stanu MIDI zapisana: {output_midi_state_path}")

    print("Nagrywanie zakończone.")

# Funkcja do przetwarzania danych landmarków (jedna ręka, dwie ręce, brak rąk)
def preprocess_landmarks(landmarks, max_hands=2):
    num_frames = len(landmarks)
    processed_landmarks = []

    for frame_landmarks in landmarks:
        frame_data = []

        # Przetwarzanie każdej dłoni w ramce
        for hand_landmarks in frame_landmarks:
            if hand_landmarks:  # Jeśli dłoń jest wykryta
                frame_data.append(hand_landmarks.flatten())
            else:  # Jeśli dłoń nie jest wykryta, wstaw zera
                frame_data.append(np.zeros(21 * 3))  # 21 punktów, każdy z 3 współrzędnymi

        # Upewnij się, że mamy dane dla max_hands rąk
        while len(frame_data) < max_hands:
            frame_data.append(np.zeros(21 * 3))

        processed_landmarks.append(np.hstack(frame_data))

    return np.array(processed_landmarks)  # (num_frames, max_hands * 21 * 3)

# Główna funkcja
if __name__ == "__main__":
    # Folder do zapisu danych
    output_folder = "C:\\Users\\Malak\\data2"
    os.makedirs(output_folder, exist_ok=True)

    # Ścieżki plików wyjściowych
    video_path = os.path.join(output_folder, "recording.avi")
    midi_state_path = os.path.join(output_folder, "midi_state.txt")

    # Rozpocznij nagrywanie
    record_video_and_midi(video_path, midi_state_path)


Rozpoczynanie nagrywania wideo i MIDI...
Historia stanu MIDI zapisana: C:\Users\Malak\data2\midi_state.txt
Nagrywanie zakończone.
